In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from scipy.spatial import KDTree

In [ ]:
data=pd.read_csv("train_data_wo_distances.csv")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6768 entries, 0 to 6767
Data columns (total 25 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   id                                                           6768 non-null   object 
 1   is_appartment                                                6768 non-null   bool   
 2   area                                                         6768 non-null   float64
 3   new_building                                                 6768 non-null   int64  
 4   postcode                                                     6768 non-null   int64  
 5   lat                                                          6768 non-null   float64
 6   lon                                                          6768 non-null   float64
 7   province                                                     6768 non-null   o

,id,is_appartment,area,new_building,postcode,lat,lon,province,bedroom_category,had_price_drop,...,is_sticker,sqm_per_bedroom,sqm_per_bedroom_bin,energy_per_sqm,energy_label_numeric,cluster_A year on the market,"cluster_Apartments, smaller properties, fresh on the market",cluster_Huge areas - Villas,cluster_More years on the market,cluster_Single-Family Homes
0,te0,False,346.0,0,9600,50.748390,3.604941,Oost-Vlaanderen,Standard (2-4 Beds),1,...,1,86.500000,Extreme,1.841040,6,0,0,0,0,1
1,te1,False,173.0,0,9890,50.937660,3.632149,Oost-Vlaanderen,Standard (2-4 Beds),0,...,0,43.250000,Average,1.190751,3,0,0,0,0,1
2,te2,True,89.0,0,4600,50.736616,5.698354,Luik,Standard (2-4 Beds),0,...,1,29.666667,Small,2.471910,3,0,1,0,0,0
3,te3,False,71.0,0,9500,50.776460,3.881396,Oost-Vlaanderen,Standard (2-4 Beds),0,...,1,17.750000,Small,2.873239,3,0,1,0,0,0
4,te4,True,124.0,0,3600,50.963370,5.494989,Limburg,Standard (2-4 Beds),0,...,0,41.333333,Average,0.766129,1,0,1,0,0,0


In [4]:
major_cities=pd.read_csv("major Belgian cities.csv")
major_cities.head()

,city,lat,lon
0,Brussels,50.850,4.349
1,Antwerp,51.220,4.400
2,Ghent,51.050,3.717
3,Charleroi,50.411,4.444
4,Liège,50.634,5.567


In [5]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    distance = R * c
    
    return distance

In [ ]:
def find_nearest_city(latitude,longitude):
    distances = []
    for city in major_cities.itertuples():
        distance = calculate_distance(latitude, longitude, city.lat, city.lon)
        distances.append(distance)
    
    return min(distances)

print(find_nearest_city(51.08453,5.108020))

data["nearest_city_distance"]= data.apply(
    lambda row: find_nearest_city(row['lat'], row['lon']), axis=1)



data.head()

36.4564897289071


,id,is_appartment,area,new_building,postcode,lat,lon,province,bedroom_category,had_price_drop,...,sqm_per_bedroom,sqm_per_bedroom_bin,energy_per_sqm,energy_label_numeric,cluster_A year on the market,"cluster_Apartments, smaller properties, fresh on the market",cluster_Huge areas - Villas,cluster_More years on the market,cluster_Single-Family Homes,nearest_city_distance
0,te0,False,346.0,0,9600,50.748390,3.604941,Oost-Vlaanderen,Standard (2-4 Beds),1,...,86.500000,Extreme,1.841040,6,0,0,0,0,1,34.445912
1,te1,False,173.0,0,9890,50.937660,3.632149,Oost-Vlaanderen,Standard (2-4 Beds),0,...,43.250000,Average,1.190751,3,0,0,0,0,1,13.831337
2,te2,True,89.0,0,4600,50.736616,5.698354,Luik,Standard (2-4 Beds),0,...,29.666667,Small,2.471910,3,0,1,0,0,0,14.691231
3,te3,False,71.0,0,9500,50.776460,3.881396,Oost-Vlaanderen,Standard (2-4 Beds),0,...,17.750000,Small,2.873239,3,0,1,0,0,0,31.179893
4,te4,True,124.0,0,3600,50.963370,5.494989,Limburg,Standard (2-4 Beds),0,...,41.333333,Average,0.766129,1,0,1,0,0,0,36.972291


In [ ]:
train_stations=pd.read_csv("belgium_train_stations.csv")

def find_nearest_station(latitude,longitude):
    distances = []
    for station in train_stations.itertuples():
        distance = calculate_distance(latitude, longitude, station.latitude, station.longitude)
        distances.append(distance)
    
    return min(distances)

print(find_nearest_station(51.08453,5.108020))

data["nearest_train_station_distance"]= data.apply(
    lambda row: find_nearest_station(row['lat'], row['lon']), axis=1)



data.head()

8.713872174907433


,id,is_appartment,area,new_building,postcode,lat,lon,province,bedroom_category,had_price_drop,...,sqm_per_bedroom_bin,energy_per_sqm,energy_label_numeric,cluster_A year on the market,"cluster_Apartments, smaller properties, fresh on the market",cluster_Huge areas - Villas,cluster_More years on the market,cluster_Single-Family Homes,nearest_city_distance,nearest_train_station_distance
0,te0,False,346.0,0,9600,50.748390,3.604941,Oost-Vlaanderen,Standard (2-4 Beds),1,...,Extreme,1.841040,6,0,0,0,0,1,34.445912,0.773084
1,te1,False,173.0,0,9890,50.937660,3.632149,Oost-Vlaanderen,Standard (2-4 Beds),0,...,Average,1.190751,3,0,0,0,0,1,13.831337,0.844822
2,te2,True,89.0,0,4600,50.736616,5.698354,Luik,Standard (2-4 Beds),0,...,Small,2.471910,3,0,1,0,0,0,14.691231,0.439527
3,te3,False,71.0,0,9500,50.776460,3.881396,Oost-Vlaanderen,Standard (2-4 Beds),0,...,Small,2.873239,3,0,1,0,0,0,31.179893,0.870566
4,te4,True,124.0,0,3600,50.963370,5.494989,Limburg,Standard (2-4 Beds),0,...,Average,0.766129,1,0,1,0,0,0,36.972291,0.533166


In [ ]:
bus_stops=pd.read_csv("belgium_bus_stops.csv")

def find_nearest_stop(latitude,longitude):
    distances = []
    for stop in bus_stops.itertuples():
        distance = calculate_distance(latitude, longitude, stop.lat, stop.lon)
        distances.append(distance)
    
    return min(distances)

print(find_nearest_stop(51.08453,5.108020))

data["nearest_bus_stop_distance"]= data.apply(
    lambda row: find_nearest_stop(row['lat'], row['lon']), axis=1)



data.head()

0.13632895008635754


,id,is_appartment,area,new_building,postcode,lat,lon,province,bedroom_category,had_price_drop,...,energy_per_sqm,energy_label_numeric,cluster_A year on the market,"cluster_Apartments, smaller properties, fresh on the market",cluster_Huge areas - Villas,cluster_More years on the market,cluster_Single-Family Homes,nearest_city_distance,nearest_train_station_distance,nearest_bus_stop_distance
0,te0,False,346.0,0,9600,50.748390,3.604941,Oost-Vlaanderen,Standard (2-4 Beds),1,...,1.841040,6,0,0,0,0,1,34.445912,0.773084,0.038041
1,te1,False,173.0,0,9890,50.937660,3.632149,Oost-Vlaanderen,Standard (2-4 Beds),0,...,1.190751,3,0,0,0,0,1,13.831337,0.844822,0.134522
2,te2,True,89.0,0,4600,50.736616,5.698354,Luik,Standard (2-4 Beds),0,...,2.471910,3,0,1,0,0,0,14.691231,0.439527,0.121519
3,te3,False,71.0,0,9500,50.776460,3.881396,Oost-Vlaanderen,Standard (2-4 Beds),0,...,2.873239,3,0,1,0,0,0,31.179893,0.870566,0.034633
4,te4,True,124.0,0,3600,50.963370,5.494989,Limburg,Standard (2-4 Beds),0,...,0.766129,1,0,1,0,0,0,36.972291,0.533166,0.149507


In [ ]:
data=data.drop(columns=['cluster_More years on the market','cluster_A year on the market', 'lat','lon'])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6768 entries, 0 to 6767
Data columns (total 24 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   id                                                           6768 non-null   object 
 1   is_appartment                                                6768 non-null   bool   
 2   area                                                         6768 non-null   float64
 3   new_building                                                 6768 non-null   int64  
 4   postcode                                                     6768 non-null   int64  
 5   province                                                     6768 non-null   object 
 6   bedroom_category                                             6768 non-null   object 
 7   had_price_drop                                               6768 non-null   i

In [ ]:
data.to_csv("train_data.csv")